In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\rem31\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


### Visit the NASA Mars News Site

In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')

In [5]:
slide_elem.find('div', class_='content_title')

<div class="content_title">Curiosity Mars Rover's Summer Road Trip Has Begun</div>

In [6]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()


In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()

### JPL Space Images Featured Image

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')


In [11]:
# find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')


In [12]:
# Use the base url to create an absolute url
img_url = f'https://spaceimages-mars.com/{img_url_rel}'


### Mars Facts

In [13]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]


In [14]:
df.columns=['Description', 'Mars', 'Earth']
df.set_index('Description', inplace=True)


In [15]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [16]:
# 1. Use browser to visit the URL 
url='https://marshemispheres.com/'
# url = 'https://astrogeology.usgs.gov/'

browser.visit(url)

In [17]:
html_hemispheres = browser.html
hemispheres_bs=soup(html_hemispheres,'html.parser')


In [18]:
results=hemispheres_bs.find('div', class_='result-list')
hemispheres=results.find_all('div',{'class':'item'})

In [19]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls=[]
hemisphere={}

# 3. Write code to retrieve the image urls and titles for each hemisphere.

for hemisphere in hemispheres:
# hemisphere=hemispheres[0]
    title = hemisphere.find("h3").text
    end_link = hemisphere.find("a")["href"]
    hemisphere_link = "https://marshemispheres.com/" + end_link    
    browser.visit(hemisphere_link)

    hemisphere_html = browser.html
    hemisphere_soup=soup(hemisphere_html, "html.parser")
    hemisphere_image_link = hemisphere_soup.find("img",{"class":"wide-image"})["src"]
    hemisphere_wide_image_link = "https://marshemispheres.com/" + hemisphere_image_link
    hemisphere_dict={
        "title":title,
        "image_url":hemisphere_wide_image_link
        }
    hemisphere_image_urls.append(hemisphere_dict)
    

In [20]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [23]:
# 5. Quit the browser
browser.quit()